# How to run this notebook
`bazel build //TransformCalculator:jupyter && ./bazel-bin/TransformCalculator/jupyter`

# Goal
We need a tool that can be used to reduce bugs when working with transforms.
We want to be able to chain transforms together, and keep track of what they do.

To do so we need two mechanisms:
- A graph data structure to link transforms together.
- A Transform class to add semantic information to mathematical transforms.

In [39]:
class Transform:
    """
    Generic transform class.
    Can be used with any transform that can be composed, 
    for instance SE3, SO3, S1, etc
    """
    
    def __init__(self, transform, to: str, frm: str):
        self.transform_type = type(transform)
        self.transform = transform
        self.to = to
        self.frm = frm
        
    def __repr__(self):
        return f"Transforms from {self.frm} to {self.to} using {self.transform}"
        
    def __matmul__(self, rhs):
        assert(self.transform_type == rhs.transform_type)
        assert(self.to == rhs.frm)
        return Transform(
            transform = self.transform @ rhs.transform,
            to = rhs.to,
            frm = self.frm)
    
    def inverse(self):
        return Transform(self.transform.inverse(), to = self.frm, frm = self.to)

In [43]:
from lib.Transforms.CircleGroup.S1 import S1
S1_orig_to_90deg = S1(1.5)
Ta = Transform(S1_orig_to_90deg, to = "top", frm = "orig")
Ta_i = Ta.inverse()
Tb = Transform(S1(0.1), frm = "top", to = "past_top")
Tc = (Ta @ Tb).inverse() @ Ta

In [88]:
print(Ta)
print(Tb)
print((Ta @ Tb).inverse())
print(Tc)

Transforms from orig to top using S1: 1.5 rads
Transforms from top to past_top using S1: 0.09999999999999999 rads
Transforms from past_top to orig using S1: -1.6 rads
Transforms from past_top to top using S1: -0.1 rads


In [155]:
from collections import defaultdict, namedtuple

class Graph:
    
    def __init__(self):
        self.nodes = defaultdict(dict)
        
    def __repr__(self):
        return f"{self.nodes}"
    
    def add_node(self, node, origin, destination):
        self.nodes[origin][destination] = node
        
    def get_node(self, origin, destination):
        return self.nodes[origin][destination]
 
    def fill_graph(self):
        # Compute every possible transform
        pass
    
    def get_global_error(self):
        # Compute inconsistencies between transforms
                                 
    def search(self, node):
        pass
    
    def find_all_paths(self, start, end, path=[]):
        graph = self.nodes
        path = path + [start]
        if start == end:
            return [path]
        if start not in graph.keys():
            return []
        paths = []
        for node in graph[start].keys():
            if node not in path:
                newpaths = self.find_all_paths(node, end, path)
                for newpath in newpaths:
                    paths.append(newpath)
        return paths
    
    def remove_node(self, node):
        pass

In [156]:
g = Graph()

for T in [Ta, Tb]:
    g.add_node(node = T, origin = T.frm, destination = T.to)
    g.add_node(node = T.inverse(), origin = T.to, destination = T.frm)

g.nodes["top"]

{'orig': Transforms from top to orig using S1: -1.5 rads,
 'past_top': Transforms from top to past_top using S1: 0.09999999999999999 rads}

In [164]:
def get_chained_transform(graph, frm, to):
    path = g.find_all_paths(start = frm, end = to)
    
    # tmp
    path = path[0]
    
    # find correct type
    transforms = []
    for i in range(1, len(path)):
        key_from = path[i-1]
        key_to = path[i]
        node = g.get_node(origin = key_from, destination = key_to)
        transforms.append(node)
                
    T = transforms[0]
    for i in range(1, len(transforms)):
        T = T @ transforms[i]
    return T

get_chained_transform(g,  frm = "orig", to = "past_top")

Transforms from orig to past_top using S1: 1.6 rads

In [165]:
g.find_all_paths(start = "past_top", end = "orig")

[['past_top', 'top', 'orig']]

In [168]:
get_chained_transform(g,  frm = "past_top", to = "orig")

Transforms from past_top to orig using S1: -1.6 rads